<a href="https://colab.research.google.com/github/DhekerKacem/OpenclassroomsProject/blob/main/projet_7_bert_colab_dhaker_(3).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Réalisez une analyse de sentiments grâce au Deep Learning

### Présentation du projet

La compagnie aérienne Air Paradis a missionné notre cabinet pour créer un produit IA permettant d’anticiper les bad buzz sur les réseaux sociaux.

Air Paradis veut un prototype d’un produit IA permettant de prédire le sentiment associé à un tweet.

### Importation des bibliothèques

In [1]:
pip install mlflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.2/20.2 MB 51.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.6/147.6 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.2/128.2 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.2/80.2 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.8/78.8 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 202.9/202.9 kB 21.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.8/52.8 kB 4.8 MB/s eta 0:00:00


In [2]:
pip install transformers

In [3]:
pip install tensorflow

In [1]:
import transformers
from transformers import BertModel, BertTokenizer, AdamW, get_linear_schedule_with_warmup
import numpy as np
from torch.utils.data import Dataset, DataLoader
from collections import defaultdict
from torch import nn, optim
import pandas as pd
import seaborn as sns
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import string
import gensim
import os
from torch import nn, optim
from torch.optim.lr_scheduler import ReduceLROnPlateau  # Import de ReduceLROnPlateau
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import LSTM, Dense, Embedding, SpatialDropout1D, Bidirectional
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from keras.layers import Dropout
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords as nltk_stopwords
from nltk.tokenize import word_tokenize, wordpunct_tokenize, RegexpTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import scipy.sparse
from transformers import DistilBertTokenizer, DistilBertModel
import torch
from transformers import RobertaTokenizer, RobertaModel
from transformers import BertTokenizer
import warnings
warnings.filterwarnings('ignore')

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Importer et explorer les données

In [3]:
data = pd.read_csv('/content/drive/My Drive/df_kaggle.csv', delimiter=',', skiprows=[5312], encoding='latin1')

In [4]:
data.head()

,Label,Text
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,is upset that he can't update his Facebook by ...
2,0,@Kenichan I dived many times for the ball. Man...
3,0,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all...."


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1581338 entries, 0 to 1581337
Data columns (total 2 columns):
 #   Column  Non-Null Count    Dtype 
---  ------  --------------    ----- 
 0   Label   1581338 non-null  int64 
 1   Text    1581338 non-null  object
dtypes: int64(1), object(1)
memory usage: 24.1+ MB


In [6]:
data.nunique()

Label          2
Text     1581338
dtype: int64

In [7]:
data.isna().sum()

Label    0
Text     0
dtype: int64

In [8]:
data.describe()

,Label
count,1.581338e+06
mean,5.003225e-01
std,5.000001e-01
min,0.000000e+00
25%,0.000000e+00
50%,1.000000e+00
75%,1.000000e+00
max,1.000000e+00


### Bert pré entrainé

### Bert, Hugging Face

In [9]:
import gc
gc.collect()

31

In [10]:
torch.cuda.empty_cache()

In [11]:
!nvidia-smi

Wed Apr 17 14:57:55 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0              50W / 400W |      2MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

### Entrainement d'un premier modèle avec un échantillion de données

### Modèle avec Early Stopping

In [12]:
# Déterminer le nombre d'observations à sélectionner dans chaque classe
num_samples_per_class = 300000

# Diviser le dataframe en deux parties selon les labels
df_label_0 = data[data['Label'] == 0]
df_label_1 = data[data['Label'] == 1]

# Sous-échantillonner chaque classe pour obtenir le nombre souhaité d'observations
df_label_0 = df_label_0.sample(n=num_samples_per_class, random_state=42)
df_label_1 = df_label_1.sample(n=num_samples_per_class, random_state=42)

# Concaténer les deux parties pour former un nouvel ensemble de données équilibré
df_balanced = pd.concat([df_label_0, df_label_1])

# Mélanger les données
df_balanced = df_balanced.sample(frac=1, random_state=42).reset_index(drop=True)

# Afficher les informations sur le DataFrame équilibré
print(df_balanced.info())

# Sélectionner les 40 000 premières observations
df_balanced_test = df_balanced.head(20000)
df_balanced_final =  df_balanced.head(500000)

# Afficher la taille des deux base de test et finale
print("Taille de la base de données de test :", df_balanced_test.shape)
print("Taille de la base de données finale :", df_balanced_final.shape)

# Calcul des fréquences des valeurs de la variable cible et tri par ordre décroissant
print("Distribution de la variable Label dans df_balanced_test:")
print(df_balanced_test['Label'].value_counts().sort_values(ascending=False))

print("Distribution de la variable Label dans df_balanced_final:")
print(df_balanced_final['Label'].value_counts().sort_values(ascending=False))


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 600000 entries, 0 to 599999
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   Label   600000 non-null  int64 
 1   Text    600000 non-null  object
dtypes: int64(1), object(1)
memory usage: 9.2+ MB
None
Taille de la base de données de test : (20000, 2)
Taille de la base de données finale : (500000, 2)
Distribution de la variable Label dans df_balanced_test:
Label
0    10017
1     9983
Name: count, dtype: int64
Distribution de la variable Label dans df_balanced_final:
Label
1    250021
0    249979
Name: count, dtype: int64


### Avec gel du modèle BERT pré-entrainé

In [13]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from transformers import BertModel, BertTokenizer, AdamW, get_linear_schedule_with_warmup
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd

# Définition du modèle BERT à utiliser
PRE_TRAINED_MODEL_NAME = 'bert-base-cased'

# Chargement du tokenizer
tokenizer = BertTokenizer.from_pretrained(PRE_TRAINED_MODEL_NAME)

# Longueur maximale des séquences
MAX_LEN = 200

# Classe Dataset pour traiter les données
class GPReviewDataset(Dataset):
    def __init__(self, reviews, targets, tokenizer, max_len):
        self.reviews = reviews
        self.targets = targets
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.reviews)

    def __getitem__(self, item):
        review = str(self.reviews[item])
        target = self.targets[item]
        encoding = self.tokenizer.encode_plus(
            review,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            pad_to_max_length=True,
            return_attention_mask=True,
            return_tensors='pt'
        )
        return {
            'input_ids': encoding['input_ids'],
            'attention_mask': encoding['attention_mask'],
            'targets': torch.tensor(target, dtype=torch.float)
        }

# Séparation des données en ensembles d'entraînement, de validation et de test
# Notez que nous utilisons ici les données df_train et df_val créées précédemment
df_train, df_test = train_test_split(df_balanced_test, test_size=0.2, random_state=42)
df_val, df_test = train_test_split(df_test, test_size=0.5, random_state=42)

# Création des DataLoaders pour le premier modèle avec 20 000 observations
BATCH_SIZE = 16

def create_data_loader(df, tokenizer, max_len, batch_size):
    ds = GPReviewDataset(
        reviews=df['Text'].to_numpy(),
        targets=df['Label'].to_numpy(),
        tokenizer=tokenizer,
        max_len=max_len
    )
    return DataLoader(ds, batch_size=batch_size)
train_data_loader = create_data_loader(df_train, tokenizer, MAX_LEN, BATCH_SIZE)
val_data_loader = create_data_loader(df_val, tokenizer, MAX_LEN, BATCH_SIZE)

# Définition du modèle de classification
class SentimentClassifier(nn.Module):
    def __init__(self, PRE_TRAINED_MODEL_NAME, freeze_bert=False):
        super(SentimentClassifier, self).__init__()
        self.bert = BertModel.from_pretrained(PRE_TRAINED_MODEL_NAME)
        self.drop = nn.Dropout(p=0.3)
        self.out = nn.Linear(self.bert.config.hidden_size, 1)  # Une seule sortie pour la classification binaire

        if freeze_bert:
            for param in self.bert.parameters():
                param.requires_grad = False

    def forward(self, input_ids, attention_mask):
        input_ids = input_ids.squeeze(1)  # Supprime la dimension supplémentaire de dimension 1
        attention_mask = attention_mask.squeeze(1)  # Supprime la dimension supplémentaire de dimension 1
        outputs = self.bert(
            input_ids=input_ids,
            attention_mask=attention_mask
        )
        pooled_output = outputs.pooler_output
        output = self.drop(pooled_output)
        output = self.out(output)
        return output.squeeze(-1)

# Création d'une instance du modèle
model_test = SentimentClassifier(PRE_TRAINED_MODEL_NAME, freeze_bert=True)

# Configuration de l'appareil
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_test = model_test.to(device)

# Nombre de paramètres à entraîner
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'Le modèle a {count_parameters(model_test):,} paramètres à entraîner')

# Définition de l'optimiseur, du scheduler et de la fonction de perte
EPOCHS = 5
optimizer = AdamW(model_test.parameters(), lr=2e-5, correct_bias=False)
total_steps = len(train_data_loader) * EPOCHS
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)
loss_fn = nn.BCEWithLogitsLoss().to(device)

# Fonction d'entraînement pour une époque
def train_epoch(model, data_loader, loss_fn, optimizer, device, scheduler, n_examples):
    model = model.train()
    losses = []
    correct_predictions = 0

    for d in data_loader:
        input_ids = d["input_ids"].to(device)
        attention_mask = d["attention_mask"].to(device)
        targets = d["targets"].to(device)

        optimizer.zero_grad()

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)

        preds = torch.round(torch.sigmoid(outputs))

        correct_predictions += torch.sum(preds == targets)

        loss = loss_fn(outputs, targets)
        losses.append(loss.item())

        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        scheduler.step()

    return correct_predictions.double() / n_examples, np.mean(losses)

# Fonction d'évaluation du modèle
def eval_model(model, data_loader, loss_fn, device, n_examples):
    model = model.eval()
    losses = []
    correct_predictions = 0

    with torch.no_grad():
        for d in data_loader:
            input_ids = d["input_ids"].to(device)
            attention_mask = d["attention_mask"].to(device)
            targets = d["targets"].to(device)

            outputs = model(input_ids=input_ids.squeeze(), attention_mask=attention_mask.squeeze())

            preds = torch.round(torch.sigmoid(outputs)).squeeze()
            correct_predictions += torch.sum(preds == targets)

            loss = loss_fn(outputs, targets)
            losses.append(loss.item())

    return correct_predictions.double() / n_examples, np.mean(losses)

# Implémentation de l'arrêt précoce
patience = 3
best_accuracy = 0
counter = 0

# Entraînement du modèle avec arrêt précoce
for epoch in range(EPOCHS):
    print(f"Epoch {epoch + 1}/{EPOCHS}")
    print('-' * 10)

    train_acc, train_loss = train_epoch(
        model_test,
        train_data_loader,
        loss_fn,
        optimizer,
        device,
        scheduler,
        len(df_train)
    )

    print(f'Train loss {train_loss} accuracy {train_acc}')

    val_acc, val_loss = eval_model(
        model_test,
        val_data_loader,
        loss_fn,
        device,
        len(df_val)
    )

    print(f'Val   loss {val_loss} accuracy {val_acc}')
    print()

    # Vérifier si la précision sur l'ensemble de validation a augmenté
    if val_acc > best_accuracy:
        best_accuracy = val_acc
        counter = 0  # Réinitialiser le compteur de patience
        torch.save(model_test.state_dict(), 'best_model_state.bin')
    else:
        counter += 1  # Incrémenter le compteur de patience

    # Vérifier si l'arrêt précoce est déclenché
    if counter >= patience:
        print("Early stopping triggered!")
        break


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Le modèle a 769 paramètres à entraîner


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Epoch 1/5
----------
Train loss 0.6969590681791306 accuracy 0.524375
Val   loss 0.6833999266624451 accuracy 0.597

Epoch 2/5
----------
Train loss 0.6929431904554367 accuracy 0.530625
Val   loss 0.6803799867630005 accuracy 0.6025

Epoch 3/5
----------
Train loss 0.6883511034250259 accuracy 0.5443125
Val   loss 0.6784353761672973 accuracy 0.6085

Epoch 4/5
----------


KeyboardInterrupt: 

In [14]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import BertModel, BertTokenizer, AdamW, get_linear_schedule_with_warmup
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd

# Définition du modèle BERT à utiliser
PRE_TRAINED_MODEL_NAME = 'bert-base-cased'

# Chargement du tokenizer
tokenizer = BertTokenizer.from_pretrained(PRE_TRAINED_MODEL_NAME)

# Longueur maximale des séquences
MAX_LEN = 200

# Classe Dataset pour traiter les données
class GPReviewDataset(Dataset):
    def __init__(self, reviews, targets, tokenizer, max_len):
        self.reviews = reviews
        self.targets = targets
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.reviews)

    def __getitem__(self, item):
        review = str(self.reviews[item])
        target = self.targets[item]
        encoding = self.tokenizer.encode_plus(
            review,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            padding='max_length',
            return_attention_mask=True,
            return_tensors='pt'
        )
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'targets': torch.tensor(target, dtype=torch.float)
        }

# Séparation des données en ensembles d'entraînement, de validation et de test
df_train, df_test = train_test_split(df_balanced_test, test_size=0.2, random_state=42)
df_val, df_test = train_test_split(df_test, test_size=0.5, random_state=42)

# Création des DataLoaders
BATCH_SIZE = 16
def create_data_loader(df, tokenizer, max_len, batch_size):
    ds = GPReviewDataset(
        reviews=df['Text'].to_numpy(),
        targets=df['Label'].to_numpy(),
        tokenizer=tokenizer,
        max_len=max_len
    )
    return DataLoader(ds, batch_size=batch_size)
train_data_loader = create_data_loader(df_train, tokenizer, MAX_LEN, BATCH_SIZE)
val_data_loader = create_data_loader(df_val, tokenizer, MAX_LEN, BATCH_SIZE)
test_data_loader = create_data_loader(df_test, tokenizer, MAX_LEN, BATCH_SIZE)


# Définition du modèle de classification
class SentimentClassifier(nn.Module):
    def __init__(self, PRE_TRAINED_MODEL_NAME, freeze_bert=False):
        super(SentimentClassifier, self).__init__()
        self.bert = BertModel.from_pretrained(PRE_TRAINED_MODEL_NAME)
        self.drop = nn.Dropout(p=0.3)
        self.out = nn.Linear(self.bert.config.hidden_size, 1)

        if freeze_bert:
            for param in self.bert.parameters():
                param.requires_grad = False

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        output = self.drop(outputs.pooler_output)
        return self.out(output).squeeze(-1)

# Configuration de l'appareil et du modèle
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_test = SentimentClassifier(PRE_TRAINED_MODEL_NAME, freeze_bert=True)
model_test = model_test.to(device)

# Nombre de paramètres à entraîner
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'Le modèle a {count_parameters(model_test):,} paramètres à entraîner')

# Définition de l'optimiseur, du scheduler et de la fonction de perte
EPOCHS = 10
optimizer = AdamW(model_test.parameters(), lr=2e-5, correct_bias=False)
total_steps = len(train_data_loader) * EPOCHS
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)
loss_fn = nn.BCEWithLogitsLoss().to(device)

# Fonctions d'entraînement et d'évaluation
def train_epoch(model, data_loader, loss_fn, optimizer, device, scheduler, n_examples):
    model.train()
    losses = []
    correct_predictions = 0
    for d in data_loader:
        input_ids = d["input_ids"].to(device)
        attention_mask = d["attention_mask"].to(device)
        targets = d["targets"].to(device)

        optimizer.zero_grad()

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        loss = loss_fn(outputs, targets)
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        scheduler.step()

        losses.append(loss.item())
        preds = torch.round(torch.sigmoid(outputs))
        correct_predictions += torch.sum(preds == targets)

    return correct_predictions.double() / n_examples, np.mean(losses)

def eval_model(model, data_loader, loss_fn, device, n_examples):
    model.eval()
    losses = []
    correct_predictions = 0
    with torch.no_grad():
        for d in data_loader:
            input_ids = d["input_ids"].to(device)
            attention_mask = d["attention_mask"].to(device)
            targets = d["targets"].to(device)

            outputs = model(input_ids=input_ids.squeeze(), attention_mask=attention_mask.squeeze())
            loss = loss_fn(outputs, targets)
            losses.append(loss.item())
            preds = torch.round(torch.sigmoid(outputs)).squeeze()
            correct_predictions += torch.sum(preds == targets)

    return correct_predictions.double() / n_examples, np.mean(losses)

# Définition du chemin de sauvegarde sur Kaggle
save_dir = '/kaggle/test/working'
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

# Entraînement avec arrêt précoce
best_accuracy = 0
counter = 0
patience = 3

for epoch in range(EPOCHS):
    print(f"Epoch {epoch + 1}/{EPOCHS}")
    train_acc, train_loss = train_epoch(
        model_test, train_data_loader, loss_fn, optimizer, device, scheduler, len(df_train)
    )
    print(f'Train loss {train_loss} accuracy {train_acc}')

    val_acc, val_loss = eval_model(
        model_test, val_data_loader, loss_fn, device, len(df_val)
    )
    print(f'Val loss {val_loss} accuracy {val_acc}')

    if val_acc > best_accuracy:
        best_accuracy = val_acc
        counter = 0
        best_model_path = os.path.join(save_dir, 'bert_model_best.bin')
        torch.save(model_test.state_dict(), best_model_path)
        print("Best model saved with accuracy:", best_accuracy)
    else:
        counter += 1

    if counter >= patience:
        print("Early stopping triggered!")
        last_model_path = os.path.join(save_dir, 'bert_model_last.bin')
        if val_acc != best_accuracy:  # Ensure to save last model if it's not the best
            torch.save(model_test.state_dict(), last_model_path)
            print("Last model state saved before stopping.")
        break

Le modèle a 769 paramètres à entraîner
Epoch 1/10
Train loss 0.6942989304065704 accuracy 0.5280625
Val loss 0.6823889117240906 accuracy 0.5855
Best model saved with accuracy: tensor(0.5855, device='cuda:0', dtype=torch.float64)
Epoch 2/10
Train loss 0.6939999147057533 accuracy 0.5299375000000001
Val loss 0.6786454081535339 accuracy 0.5945
Best model saved with accuracy: tensor(0.5945, device='cuda:0', dtype=torch.float64)
Epoch 3/10
Train loss 0.6865101940631867 accuracy 0.552625
Val loss 0.6758310317993164 accuracy 0.6015
Best model saved with accuracy: tensor(0.6015, device='cuda:0', dtype=torch.float64)
Epoch 4/10
Train loss 0.6852405787110328 accuracy 0.5498125
Val loss 0.6736186800003052 accuracy 0.608
Best model saved with accuracy: tensor(0.6080, device='cuda:0', dtype=torch.float64)
Epoch 5/10
Train loss 0.6857095863223076 accuracy 0.5504375
Val loss 0.6717631516456604 accuracy 0.612
Best model saved with accuracy: tensor(0.6120, device='cuda:0', dtype=torch.float64)
Epoch 6/10

### Sans gel du modèle BERT pré-entrainé

In [15]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# Définition du chemin de sauvegarde sur Google Drive
save_directory_test = '/content/drive/My Drive/Colab Models test'

import os
if not os.path.exists(save_directory_test):
    os.makedirs(save_directory_test)

Mounted at /content/drive


In [16]:
# Définition du modèle BERT à utiliser
PRE_TRAINED_MODEL_NAME = 'bert-base-cased'

# Chargement du tokenizer
tokenizer = BertTokenizer.from_pretrained(PRE_TRAINED_MODEL_NAME)

# Spécifiez le répertoire où vous voulez sauvegarder le tokenizer
tokenizer.save_pretrained(save_directory_test)

# Longueur maximale des séquences
MAX_LEN = 200

# Classe Dataset pour traiter les données
class GPReviewDataset(Dataset):
    def __init__(self, reviews, targets, tokenizer, max_len):
        self.reviews = reviews
        self.targets = targets
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.reviews)

    def __getitem__(self, item):
        review = str(self.reviews[item])
        target = self.targets[item]
        encoding = self.tokenizer.encode_plus(
            review,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            padding='max_length',
            return_attention_mask=True,
            return_tensors='pt'
        )
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'targets': torch.tensor(target, dtype=torch.float)
        }

# Séparation des données en ensembles d'entraînement, de validation et de test
df_train, df_test = train_test_split(df_balanced_test, test_size=0.2, random_state=42)
df_val, df_test = train_test_split(df_test, test_size=0.5, random_state=42)

# Création des DataLoaders
BATCH_SIZE = 16
def create_data_loader(df, tokenizer, max_len, batch_size):
    ds = GPReviewDataset(
        reviews=df['Text'].to_numpy(),
        targets=df['Label'].to_numpy(),
        tokenizer=tokenizer,
        max_len=max_len
    )
    return DataLoader(ds, batch_size=batch_size)
train_data_loader = create_data_loader(df_train, tokenizer, MAX_LEN, BATCH_SIZE)
val_data_loader = create_data_loader(df_val, tokenizer, MAX_LEN, BATCH_SIZE)

# Définition du modèle de classification
class SentimentClassifier(nn.Module):
    def __init__(self):
        super(SentimentClassifier, self).__init__()
        self.bert = BertModel.from_pretrained(PRE_TRAINED_MODEL_NAME)
        self.drop = nn.Dropout(p=0.3)
        self.out = nn.Linear(self.bert.config.hidden_size, 1)

        # Sauvegarde de la configuration du modèle dans le répertoire de travail sur Google Drive
        self.config_path = os.path.join(save_directory_test, 'config.json')
        self.bert.config.save_pretrained(save_directory_test)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        output = self.drop(outputs.pooler_output)
        return self.out(output).squeeze(-1)

# Configuration de l'appareil et du modèle
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_test_sans_gel = SentimentClassifier()
model_test_sans_gel = model_test_sans_gel.to(device)

# Nombre de paramètres à entraîner
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'Le modèle a {count_parameters(model_test_sans_gel):,} paramètres à entraîner')

# Définition de l'optimiseur, du scheduler et de la fonction de perte
EPOCHS = 5
optimizer = AdamW(model_test_sans_gel.parameters(), lr=2e-5, correct_bias=False)
total_steps = len(train_data_loader) * EPOCHS
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)
loss_fn = nn.BCEWithLogitsLoss().to(device)

# Fonctions d'entraînement et d'évaluation
def train_epoch(model, data_loader, loss_fn, optimizer, device, scheduler, n_examples):
    model.train()
    losses = []
    correct_predictions = 0
    for d in data_loader:
        input_ids = d["input_ids"].to(device)
        attention_mask = d["attention_mask"].to(device)
        targets = d["targets"].to(device)

        optimizer.zero_grad()

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        loss = loss_fn(outputs, targets)
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        scheduler.step()

        losses.append(loss.item())
        preds = torch.round(torch.sigmoid(outputs))
        correct_predictions += torch.sum(preds == targets)

    return correct_predictions.double() / n_examples, np.mean(losses)

def eval_model(model, data_loader, loss_fn, device, n_examples):
    model.eval()
    losses = []
    correct_predictions = 0
    with torch.no_grad():
        for d in data_loader:
            input_ids = d["input_ids"].to(device)
            attention_mask = d["attention_mask"].to(device)
            targets = d["targets"].to(device)

            outputs = model(input_ids=input_ids.squeeze(), attention_mask=attention_mask.squeeze())
            loss = loss_fn(outputs, targets)
            losses.append(loss.item())
            preds = torch.round(torch.sigmoid(outputs)).squeeze()
            correct_predictions += torch.sum(preds == targets)

    return correct_predictions.double() / n_examples, np.mean(losses)

# Entraînement avec arrêt précoce
best_accuracy = 0
counter = 0
patience = 3

for epoch in range(EPOCHS):
    print(f"Epoch {epoch + 1}/{EPOCHS}")
    train_acc, train_loss = train_epoch(
        model_test_sans_gel, train_data_loader, loss_fn, optimizer, device, scheduler, len(df_train)
    )
    print(f'Train loss {train_loss} accuracy {train_acc}')

    val_acc, val_loss = eval_model(
        model_test_sans_gel, val_data_loader, loss_fn, device, len(df_val)
    )
    print(f'Val loss {val_loss} accuracy {val_acc}')

    if val_acc > best_accuracy:
        best_accuracy = val_acc
        counter = 0
        best_model_path = os.path.join(save_directory_test, 'bert_model_best_sans_gel.bin')
        torch.save(model_test_sans_gel.state_dict(), best_model_path)
        print("Best model saved with accuracy:", best_accuracy)
    else:
        counter += 1

    if counter >= patience:
        print("Early stopping triggered!")
        last_model_path = os.path.join(save_directory_test, 'bert_model_last_sans_gel.bin')
        if val_acc != best_accuracy:  # Ensure to save last model if it's not the best
            torch.save(model_test_sans_gel.state_dict(), last_model_path)
            print("Last model state saved before stopping.")
        break

Le modèle a 108,311,041 paramètres à entraîner
Epoch 1/5
Train loss 0.5144911636263132 accuracy 0.7455
Val loss 0.3983567134141922 accuracy 0.8195
Best model saved with accuracy: tensor(0.8195, device='cuda:0', dtype=torch.float64)
Epoch 2/5
Train loss 0.3501617063358426 accuracy 0.855625
Val loss 0.4289523103833199 accuracy 0.8245
Best model saved with accuracy: tensor(0.8245, device='cuda:0', dtype=torch.float64)
Epoch 3/5
Train loss 0.2346795699847862 accuracy 0.9185625000000001
Val loss 0.5212874783277511 accuracy 0.8270000000000001
Best model saved with accuracy: tensor(0.8270, device='cuda:0', dtype=torch.float64)
Epoch 4/5
Train loss 0.16833560051675886 accuracy 0.9530625
Val loss 0.6997864347994327 accuracy 0.8205
Epoch 5/5
Train loss 0.12328065512422472 accuracy 0.968375
Val loss 0.8044396907389164 accuracy 0.8195


Le modèle retenu est le modèle "model_test" car il fournit de meilleurs résultats et évite l'overfitting.
L'objectif maintenant est de l'entrainer sur une plus grande partie des données pour plus de robustesse. Mon choix est de l'entrainer sur 500k données car au delà cela pose un probléme de temmps de clacul même en achetant des unités de calculs permettant d'utiliser des GPU plus puissants(V100 et A100).

### Libérer de la mémoire

In [17]:
import gc
gc.collect()

90

In [18]:
torch.cuda.empty_cache()

In [19]:
!nvidia-smi

Wed Apr 17 15:24:59 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0              58W / 400W |   3707MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

### Entrainement du modèle avec l'ensemble des données

In [20]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# Définition du chemin de sauvegarde sur Google Drive
save_directory = '/content/drive/My Drive/Colab Models'

import os
if not os.path.exists(save_directory):
    os.makedirs(save_directory)

Mounted at /content/drive


In [21]:
# Définition du modèle BERT à utiliser
PRE_TRAINED_MODEL_NAME = 'bert-base-cased'

# Chargement du tokenizer
tokenizer = BertTokenizer.from_pretrained(PRE_TRAINED_MODEL_NAME)

# Spécifiez le répertoire où vous voulez sauvegarder le tokenizer
tokenizer.save_pretrained(save_directory)

# Longueur maximale des séquences
MAX_LEN = 200

# Classe Dataset pour traiter les données
class GPReviewDataset(Dataset):
    def __init__(self, reviews, targets, tokenizer, max_len):
        self.reviews = reviews
        self.targets = targets
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.reviews)

    def __getitem__(self, item):
        review = str(self.reviews[item])
        target = self.targets[item]
        encoding = self.tokenizer.encode_plus(
            review,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            padding='max_length',
            return_attention_mask=True,
            return_tensors='pt'
        )
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'targets': torch.tensor(target, dtype=torch.float)
        }

# Séparation des données en ensembles d'entraînement, de validation et de test
df_train, df_test = train_test_split(df_balanced_final, test_size=0.2, random_state=42)
df_val, df_test = train_test_split(df_test, test_size=0.5, random_state=42)

# Création des DataLoaders
BATCH_SIZE = 16
def create_data_loader(df, tokenizer, max_len, batch_size):
    ds = GPReviewDataset(
        reviews=df['Text'].to_numpy(),
        targets=df['Label'].to_numpy(),
        tokenizer=tokenizer,
        max_len=max_len
    )
    return DataLoader(ds, batch_size=batch_size)
train_data_loader = create_data_loader(df_train, tokenizer, MAX_LEN, BATCH_SIZE)
val_data_loader = create_data_loader(df_val, tokenizer, MAX_LEN, BATCH_SIZE)
test_data_loader = create_data_loader(df_test, tokenizer, MAX_LEN, BATCH_SIZE)


# Définition du modèle de classification
class SentimentClassifier(nn.Module):
    def __init__(self, PRE_TRAINED_MODEL_NAME, freeze_bert=False):
        super(SentimentClassifier, self).__init__()
        self.bert = BertModel.from_pretrained(PRE_TRAINED_MODEL_NAME)
        self.drop = nn.Dropout(p=0.3)
        self.out = nn.Linear(self.bert.config.hidden_size, 1)

        # Sauvegarde de la configuration du modèle dans le répertoire de travail sur Google Drive
        self.config_path = os.path.join(save_directory, 'config.json')
        self.bert.config.save_pretrained(save_directory)

        if freeze_bert:
            for param in self.bert.parameters():
                param.requires_grad = False

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        output = self.drop(outputs.pooler_output)
        return self.out(output).squeeze(-1)

# Configuration de l'appareil et du modèle
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_final = SentimentClassifier(PRE_TRAINED_MODEL_NAME, freeze_bert=True)
model_final = model_final.to(device)

# Nombre de paramètres à entraîner
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'Le modèle a {count_parameters(model_final):,} paramètres à entraîner')

# Définition de l'optimiseur, du scheduler et de la fonction de perte
EPOCHS = 5
optimizer = AdamW(model_final.parameters(), lr=2e-5, correct_bias=False)
total_steps = len(train_data_loader) * EPOCHS
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)
loss_fn = nn.BCEWithLogitsLoss().to(device)

# Fonctions d'entraînement et d'évaluation
def train_epoch(model, data_loader, loss_fn, optimizer, device, scheduler, n_examples):
    model.train()
    losses = []
    correct_predictions = 0
    for d in data_loader:
        input_ids = d["input_ids"].to(device)
        attention_mask = d["attention_mask"].to(device)
        targets = d["targets"].to(device)

        optimizer.zero_grad()

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        loss = loss_fn(outputs, targets)
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        scheduler.step()

        losses.append(loss.item())
        preds = torch.round(torch.sigmoid(outputs))
        correct_predictions += torch.sum(preds == targets)

    return correct_predictions.double() / n_examples, np.mean(losses)

def eval_model(model, data_loader, loss_fn, device, n_examples):
    model.eval()
    losses = []
    correct_predictions = 0
    with torch.no_grad():
        for d in data_loader:
            input_ids = d["input_ids"].to(device)
            attention_mask = d["attention_mask"].to(device)
            targets = d["targets"].to(device)

            outputs = model(input_ids=input_ids.squeeze(), attention_mask=attention_mask.squeeze())
            loss = loss_fn(outputs, targets)
            losses.append(loss.item())
            preds = torch.round(torch.sigmoid(outputs)).squeeze()
            correct_predictions += torch.sum(preds == targets)

    return correct_predictions.double() / n_examples, np.mean(losses)


# Entraînement avec arrêt précoce
best_accuracy = 0
counter = 0
patience = 2

for epoch in range(EPOCHS):
    print(f"Epoch {epoch + 1}/{EPOCHS}")
    train_acc, train_loss = train_epoch(
        model_final, train_data_loader, loss_fn, optimizer, device, scheduler, len(df_train)
    )
    print(f'Train loss {train_loss} accuracy {train_acc}')

    val_acc, val_loss = eval_model(
        model_final, val_data_loader, loss_fn, device, len(df_val)
    )
    print(f'Val loss {val_loss} accuracy {val_acc}')

    if val_acc > best_accuracy:
        best_accuracy = val_acc
        counter = 0
        best_model_path = os.path.join(save_directory, 'bert_model_best_final.bin')
        torch.save(model_final.state_dict(), best_model_path)
        print("Best model saved with accuracy:", best_accuracy)
    else:
        counter += 1

    if counter >= patience:
        print("Early stopping triggered!")
        last_model_path = os.path.join(save_directory, 'bert_model_last_final.bin')
        if val_acc != best_accuracy:  # Ensure to save last model if it's not the best
            torch.save(model_final.state_dict(), last_model_path)
            print("Last model state saved before stopping.")
        break

Le modèle a 769 paramètres à entraîner
Epoch 1/5
Train loss 0.6650820820379257 accuracy 0.5949850000000001
Val loss 0.6323787490463257 accuracy 0.67696
Best model saved with accuracy: tensor(0.6770, device='cuda:0', dtype=torch.float64)
Epoch 2/5
Train loss 0.6362748848056793 accuracy 0.6429450000000001
Val loss 0.6130142720985413 accuracy 0.69054
Best model saved with accuracy: tensor(0.6905, device='cuda:0', dtype=torch.float64)
Epoch 3/5
Train loss 0.6256007359969616 accuracy 0.6548375000000001
Val loss 0.603353365764618 accuracy 0.69842
Best model saved with accuracy: tensor(0.6984, device='cuda:0', dtype=torch.float64)
Epoch 4/5
Train loss 0.6203435582184792 accuracy 0.6584525000000001
Val loss 0.5987839398670196 accuracy 0.70096
Best model saved with accuracy: tensor(0.7010, device='cuda:0', dtype=torch.float64)
Epoch 5/5
Train loss 0.6187171956133842 accuracy 0.660945
Val loss 0.5974936360740661 accuracy 0.70186
Best model saved with accuracy: tensor(0.7019, device='cuda:0', dty

In [10]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
from transformers import BertTokenizer

PRE_TRAINED_MODEL_NAME = 'bert-base-cased'
tokenizer = BertTokenizer.from_pretrained(PRE_TRAINED_MODEL_NAME)

save_directory = '/content/drive/My Drive/Colab Models'
tokenizer.save_pretrained(save_directory)


('/content/drive/My Drive/Colab Models/tokenizer_config.json',
 '/content/drive/My Drive/Colab Models/special_tokens_map.json',
 '/content/drive/My Drive/Colab Models/vocab.txt',
 '/content/drive/My Drive/Colab Models/added_tokens.json')

### Test du modèle finale

In [21]:
# Evaluation du modèle sur l'ensemble de test
test_acc, test_loss = eval_model(
    model_final,
    test_data_loader,
    loss_fn,
    device,
    len(df_test)
)

print(f'Test   loss {test_loss} accuracy {test_acc}')

Test   loss 0.5983138734436035 accuracy 0.7012200000000001


In [2]:
def test_tweets(model, tokenizer, tweets, device, max_len=200):
    model.eval()  # Met le modèle en mode évaluation
    for tweet in tweets:
        encoded_input = tokenizer.encode_plus(
            tweet,
            add_special_tokens=True,
            max_length=max_len,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )
        input_ids = encoded_input['input_ids'].to(device)
        attention_mask = encoded_input['attention_mask'].to(device)

        with torch.no_grad():
            output = model(input_ids, attention_mask)
            prediction = torch.sigmoid(output).item()  # Assumant que le modèle retourne un score de logit pour la classification binaire

        print(f"Tweet: '{tweet}'")
        print(f"Sentiment: {'Positive' if prediction >= 0.5 else 'Negative'}\n")


In [23]:
# Charger le modèle entraîné
model_final = SentimentClassifier(PRE_TRAINED_MODEL_NAME, freeze_bert=True)
model_path = '/content/drive/My Drive/Colab Models/bert_model_best_final.bin'
model_final.load_state_dict(torch.load(model_path))
model_final = model_final.to(device)

# Charger le tokenizer BERT
tokenizer = BertTokenizer.from_pretrained(PRE_TRAINED_MODEL_NAME)

# Définir la longueur maximale des séquences
MAX_LEN = 200

# Exemple de tweets en anglais à tester
tweets_à_tester = [
    "I absolutely loved this new movie, it was really great!",
    "This restaurant was horrible, the service was very slow, and the food was cold.",
    "The new smartphone from this brand is amazing, with an excellent camera and long battery life.",
    "I'm very disappointed with this product, it doesn't work as advertised."
]

# Tester les tweets
test_tweets(model_final, tokenizer, tweets_à_tester, device)

Tweet: 'I absolutely loved this new movie, it was really great!'
Sentiment: Positive

Tweet: 'This restaurant was horrible, the service was very slow, and the food was cold.'
Sentiment: Negative

Tweet: 'The new smartphone from this brand is amazing, with an excellent camera and long battery life.'
Sentiment: Positive

Tweet: 'I'm very disappointed with this product, it doesn't work as advertised.'
Sentiment: Negative

